# Deux labo qui trouvent un résultat contradictoire

In [57]:
#importation package
import pandas as pd
import duckdb

#config affichage
pd.set_option("display.max_columns", None)  # Affiche toutes les colonnes sans limitation
pd.set_option("display.max_colwidth", None)  # Affiche la largeur complète des colonnes sans couper le texte
pd.set_option("display.expand_frame_repr", False)  # Empêche le repliement des colonnes lors de l'affichage du DataFrame

# Connexion
from pipelines.tasks.config.common import DUCKDB_FILE
con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

# Liste des tables
con.sql("SHOW TABLES").show()

┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ ana__resultats_communes │
│ cog_communes            │
│ edc_communes            │
│ edc_prelevements        │
│ edc_resultats           │
│ laposte_communes        │
│ mapping_categories      │
│ stg_communes__cog       │
│ stg_communes__laposte   │
│ stg_edc__communes       │
│ stg_edc__prevelevements │
│ stg_edc__resultats      │
├─────────────────────────┤
│         12 rows         │
└─────────────────────────┘



In [ ]:
#affichage edc_resultat avec plusieurs résultats
query_resultats_plusieurs = """ 
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux
 """
resultats_plusieurs = con.sql(query_resultats_plusieurs).df()
resultats_plusieurs

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,referenceprel,cdparametresiseeaux,nbanalyse
0,00100139398,ASP,3
1,00100139398,ODQ,3
2,00100139917,ASP,3
3,00100139917,ODQ,3
4,00100139918,ASP,3
...,...,...,...
104195,97600032328,TURBNFU,2
104196,97600032332,PH,2
104197,97600032332,TURBNFU,2
104198,97600032333,PH,2


### Explo 00100139398

In [65]:
#Explo 00100139398
query_explo_00100139398 = """ 
SELECT cdparametresiseeaux, de_ingestion_date, COUNT (cdparametresiseeaux) nbanalyse
FROM edc_resultats
WHERE referenceprel = '00100139398'
GROUP BY cdparametresiseeaux, de_ingestion_date
ORDER BY nbanalyse DESC

 """
explo_00100139398 = con.sql(query_explo_00100139398).df()
explo_00100139398

,cdparametresiseeaux,de_ingestion_date,nbanalyse
0,ASP,2025-02-28,3
1,ODQ,2025-02-28,3
2,CTF,2025-02-28,1
3,AS,2025-02-28,1
4,CDT25,2025-02-28,1
5,CL2LIB,2025-02-28,1
6,BSIR,2025-02-28,1
7,ECOLI,2025-02-28,1
8,GT22_68,2025-02-28,1
9,PH,2025-02-28,1


In [66]:
#Explo 00100139398 (détails)
query_explo_00100139398_details = """ 
SELECT *
FROM edc_resultats
WHERE referenceprel = '00100139398' AND cdparametresiseeaux IN ('ASP','ODQ')
ORDER BY cdparametresiseeaux
"""
explo_00100139398_details = con.sql(query_explo_00100139398_details).df()
explo_00100139398_details

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100139398,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148193,2024,2025-02-28,20250203-075802
1,001,00100139398,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148194,2024,2025-02-28,20250203-075802
2,001,00100139398,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148195,2024,2025-02-28,20250203-075802
3,001,00100139398,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148193,2024,2025-02-28,20250203-075802
4,001,00100139398,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148194,2024,2025-02-28,20250203-075802
5,001,00100139398,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148195,2024,2025-02-28,20250203-075802


### Explo 00100139917

In [67]:
#Explo 00100139917 (détails)
query_explo_00100139917_details = """ 
SELECT *
FROM edc_resultats
WHERE referenceprel = '00100139917' AND cdparametresiseeaux IN ('ASP','ODQ')
ORDER BY cdparametresiseeaux
"""
explo_00100139917_details = con.sql(query_explo_00100139917_details).df()
explo_00100139917_details

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date,de_dataset_datetime
0,001,00100139917,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148727,2024,2025-02-28,20250203-075802
1,001,00100139917,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148728,2024,2025-02-28,20250203-075802
2,001,00100139917,ASP,6489,ASPECT (QUALITATIF),Aspect (qualitatif),None,O,L,Aspect normal,SANS OBJET,X,None,None,0.0,None,00100148729,2024,2025-02-28,20250203-075802
3,001,00100139917,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148727,2024,2025-02-28,20250203-075802
4,001,00100139917,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148728,2024,2025-02-28,20250203-075802
5,001,00100139917,ODQ,5901,ODEUR (QUALITATIF),Odeur (qualitatif),None,O,L,Aucun changement anormal,SANS OBJET,X,None,None,0.0,None,00100148729,2024,2025-02-28,20250203-075802


### Explo sur des analyses de résultats différents

In [104]:
#affichage edc_resultat avec plusieurs résultats
query_resultat_valtraduite_diff = """ 
WITH plusieurs_resultats AS (
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux)
,
resultat_diff AS (
SELECT referenceprel, referenceanl, cdparametresiseeaux, insituana, de_ingestion_date,
LAG(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_1, 
valtraduite AS valtraduite_0, 
LEAD(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_2,
FROM edc_resultats
WHERE referenceprel IN (SELECT DISTINCT referenceprel
                        FROM plusieurs_resultats)
ORDER BY referenceprel,cdparametresiseeaux)

SELECT *
FROM resultat_diff
WHERE (valtraduite_2 AND valtraduite_1) IS NOT NULL
ORDER BY referenceprel,cdparametresiseeaux

 """
resultat_valtraduite_diff = con.sql(query_resultat_valtraduite_diff).df()
resultat_valtraduite_diff

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,referenceprel,referenceanl,cdparametresiseeaux,insituana,de_ingestion_date,valtraduite_1,valtraduite_0,valtraduite_2
0,00100138503,00100147233,ASP,L,2025-02-28,NaN,0.00,0.0
1,00100138503,00100147232,ASP,L,2025-02-28,0.0,0.00,NaN
2,00100138503,00100147232,ODQ,L,2025-02-28,NaN,0.00,0.0
3,00100138503,00100147233,ODQ,L,2025-02-28,0.0,0.00,NaN
4,00100139037,00100147813,ASP,L,2025-02-28,NaN,0.00,0.0
...,...,...,...,...,...,...,...,...
14985,97600032305,None,TURBNFU,T,2025-02-28,0.0,0.18,NaN
14986,97600032322,None,TURBNFU,T,2025-02-28,0.0,0.12,NaN
14987,97600032323,None,TURBNFU,T,2025-02-28,NaN,0.20,0.0
14988,97600032324,None,TURBNFU,T,2025-02-28,0.0,0.16,NaN


### Explo global des résultats différents

In [ ]:
#Explo global resultats différent
query_explo_global_resultat = """ 
WITH
plusieurs_resultats AS (
SELECT referenceprel, cdparametresiseeaux, COUNT(cdparametresiseeaux) AS nbanalyse
FROM edc_resultats
GROUP BY referenceprel, cdparametresiseeaux
HAVING nbanalyse >1
ORDER BY nbanalyse DESC, referenceprel, cdparametresiseeaux)
,
resultat_diff AS (
SELECT referenceprel, referenceanl, cdparametresiseeaux, insituana, de_ingestion_date,
LAG(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_1, 
valtraduite AS valtraduite_0, 
LEAD(valtraduite ) OVER(PARTITION BY referenceprel, cdparametresiseeaux ORDER BY referenceprel,cdparametresiseeaux ) AS valtraduite_2,
FROM edc_resultats
WHERE referenceprel IN (SELECT DISTINCT referenceprel
                        FROM plusieurs_resultats)
ORDER BY referenceprel,cdparametresiseeaux)
,
resultats_diff_analyse AS (
SELECT *
FROM resultat_diff
WHERE (valtraduite_2 AND valtraduite_1) IS NOT NULL
ORDER BY referenceprel,cdparametresiseeaux)


SELECT er.*
FROM resultats_diff_analyse AS rd
LEFT JOIN edc_resultats AS er
    ON rd.referenceprel = er.referenceprel AND (rd.referenceanl = er.referenceanl OR rd.referenceanl IS NULL OR er.referenceanl IS NULL)  AND rd.cdparametresiseeaux = er.cdparametresiseeaux
ORDER BY er.referenceprel, er.cdparametresiseeaux



"""
explo_global_resultat = con.sql(query_explo_global_resultat).df()
explo_global_resultat

In [ ]:
00100147232
00100147233
00100151965
05500121887


SELECT referenceanl,  COUNT (referenceanl) 
FROM selection_referenceanl
GROUP BY referenceanl
HAVING COUNT(referenceanl)>1

SELECT er.*
FROM resultats_diff_analyse AS rd
LEFT JOIN edc_resultats AS er
    ON rd.referenceprel = er.referenceprel AND rd.referenceanl = er.referenceanl AND rd.cdparametresiseeaux = er.cdparametresiseeaux
ORDER BY er.referenceprel, er.cdparametresiseeaux

97600032305

#### Détails 00100151965

In [135]:
query_resultats = """ 
SELECT DISTINCT insituana
FROM edc_resultats
WHERE referenceanl IS NULL
ORDER BY cdparametresiseeaux

 """

resultats = con.sql(query_resultats).df()
resultats

,insituana
0,T


#### conclusion
insitu T ==> pas de référenceanl

